# Ollama PDF RAG Notebook

## Import Libraries


In [1]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports
from IPython.display import display, Markdown

# Set environment variable for protobuf
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

## Load PDF

In [23]:
# Load PDF
local_path = "D:/Progect/AI_assistant/data/pdfs/OGR.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: D:/Progect/AI_assistant/data/pdfs/OGR.pdf


## Split text into chunks

In [17]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 257 chunks


In [25]:
# Assuming 'chunks' is already defined from your previous code
for i, chunk in enumerate(chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk.page_content)
    print("\n") # Add a newline for better readability between chunks

--- Chunk 1 ---
проводить инструктаж персонала Службы (подземных участков) по правилам безопасности при ведении горных работ в удароопасных условиях и проверять знание этих правил, а также назначать ответственного за проведение противоударных мероприятий в выработках с категорией "Опасно".

961. Начальник Службы имеет право:

требовать от руководителей подразделений предприятий выполнения мероприятий по

безопасности ведения горных работ на удароопасных участках;

приостанавливать горные работы при невыполнении противоударных мероприятий и

ставить в известность руководство предприятия (организации);

принимать участие в рассмотрении вопросов проведения горно-экспериментальных и исследовательских работ, направленных на разработку методов прогноза и способов борьбы с горными ударами.

962. Начальник Службы несет ответственность за:

осуществление контроля за выполнением утвержденного плана мероприятий по

предотвращению горных ударов;

состояние, ведение и хранение документации по прово

In [39]:
# Imports (без изменений)
import re
import csv
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document

# Suppress warnings (без изменений)
import warnings
warnings.filterwarnings('ignore')

# Jupyter-specific imports (без изменений)
from IPython.display import display, Markdown

# Set environment variable for protobuf (без изменений)
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# --- Функция для разбиения по порядковому номеру (без изменений, но может быть не нужна) ---
# Если вам нужно только разбиение по заголовкам, эта функция может быть удалена или не использоваться.
def split_text_by_numbered_sections(doc_list):
    """
    Разбивает каждый Document в списке по порядковым номерам (например, 407., 408.).
    Возвращает список новых Document, где каждый представляет собой пронумерованный раздел.
    Номер раздела не включается в page_content, но сохраняется в metadata.
    """
    numbered_docs = []
    # Изменено: Используем нежадный квантификатор *? для более точного соответствия
    pattern = re.compile(r'\n(?=\d+\.\s*)')

    for doc in doc_list:
        content = doc.page_content
        sections = pattern.split(content)

        current_section_text = ""
        current_section_number = None

        for section_text in sections:
            # Ищем начало нового раздела (номер. текст)
            # Захватываем номер в группу 1 и текст в группу 2
            match = re.match(r'^(\d+)\.\s*(.*)', section_text.strip(), re.DOTALL)
            if match:
                # Если у нас уже был накопленный текст для предыдущего раздела,
                # добавляем его в список numbered_docs
                if current_section_number is not None and current_section_text:
                    numbered_docs.append(Document(
                        page_content=current_section_text.strip(),
                        metadata={"section_number": current_section_number, **doc.metadata}
                    ))
                
                # Начинаем новый раздел:
                current_section_number = match.group(1) # Получаем номер раздела
                # !!! ИЗМЕНЕНИЕ ЗДЕСЬ !!!
                # Используем match.group(2) для получения только содержимого, без номера
                current_section_text = match.group(2).strip() # Содержимое нового раздела
            elif current_section_text:
                # Если это не начало нового раздела, добавляем текст к текущему
                current_section_text += "\n" + section_text.strip()

        # Добавляем последний собранный раздел после завершения цикла
        if current_section_number is not None and current_section_text:
            numbered_docs.append(Document(
                page_content=current_section_text.strip(),
                metadata={"section_number": current_section_number, **doc.metadata}
            ))
    return numbered_docs

# --- Новая функция для разбиения по заголовкам ---
def split_text_by_headings(doc_list):
    """
    Разбивает каждый Document в списке по заголовкам, начинающимся на "Требования".
    Возвращает список новых Document, где каждый представляет собой раздел под заголовком.
    Заголовок включается в page_content, но также сохраняется в metadata.
    """
    heading_docs = []
    # Паттерн для заголовков: "Требования" в начале строки,
    # затем любой текст до следующего заголовка или конца документа.
    # Используем re.DOTALL, чтобы '.' соответствовал переносам строк.
    # Используем нежадный квантификатор *? для текста после заголовка,
    # чтобы он не "забирал" следующие заголовки.
    # re.IGNORECASE для совпадения без учета регистра, если заголовки могут быть разными.
    pattern = re.compile(r'(^Требования.*?)(?=\nТребования|\Z)', re.DOTALL | re.MULTILINE)

    for doc in doc_list:
        content = doc.page_content
        
        # Находим все совпадения с паттерном заголовков
        matches = list(pattern.finditer(content))
        
        if not matches:
            # Если заголовков не найдено, добавляем весь документ как один чанк
            heading_docs.append(Document(
                page_content=content.strip(),
                metadata={"heading": "No specific heading found", **doc.metadata}
            ))
            continue

        for i, match in enumerate(matches):
            section_text = match.group(1).strip()
            
            # Извлекаем сам заголовок (первую строку)
            heading_match = re.match(r'^(Требования.*?)\n', section_text, re.DOTALL)
            heading_title = heading_match.group(1).strip() if heading_match else "Unnamed Section"

            heading_docs.append(Document(
                page_content=section_text,
                metadata={"heading": heading_title, **doc.metadata}
            ))
            
    return heading_docs

# --- Остальной код ---
# Load PDF
local_path = "D:/Progect/AI_assistant/data/pdfs/OGR.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

# --- Шаг 1: Разбить текст на чанки по заголовкам ---
# Теперь используем новую функцию split_text_by_headings
heading_chunks_data = split_text_by_headings(data)
print(f"Text initially split into {len(heading_chunks_data)} heading-based chunks.")

# --- Шаг 2 (опционально): Применить RecursiveCharacterTextSplitter к каждому из этих чанков ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=200)

final_chunks = []
for chunk_doc in heading_chunks_data:
    sub_chunks = text_splitter.split_documents([chunk_doc])
    final_chunks.extend(sub_chunks)

print(f"After further splitting (if needed), total chunks: {len(final_chunks)}")


# --- Сохранение чанков в CSV ---
output_csv_path = "D:/Progect/AI_assistant/data/chunks.csv" # Укажите желаемый путь и имя файла

try:
    with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        # Обновляем заголовки столбцов для включения 'heading' вместо 'section_number'
        fieldnames = ['chunk_id', 'page_content', 'heading', 'source_page', 'metadata']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader() # Записываем заголовки

        for i, chunk in enumerate(final_chunks):
            # Извлекаем метаданные
            heading = chunk.metadata.get('heading', 'N/A') # Используем 'heading'
            source_page = chunk.metadata.get('page', 'N/A')
            
            other_metadata = {k: v for k, v in chunk.metadata.items() if k not in ['heading', 'page', 'section_number']} # Учитываем удаление 'section_number'
            
            writer.writerow({
                'chunk_id': i,
                'page_content': chunk.page_content,
                'heading': heading, # Записываем заголовок
                'source_page': source_page,
                'metadata': str(other_metadata)
            })
    print(f"Chunks successfully saved to {output_csv_path}")
except Exception as e:
    print(f"Error saving chunks to CSV: {e}")

PDF loaded successfully: D:/Progect/AI_assistant/data/pdfs/OGR.pdf
Text initially split into 19 heading-based chunks.
After further splitting (if needed), total chunks: 47
Chunks successfully saved to D:/Progect/AI_assistant/data/chunks.csv


## Create vector database

In [40]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=final_chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")
# Добавлено: Вывод количества чанков в векторной базе данных
print(f"Number of chunks in vector database: {len(final_chunks)}")

Vector database created successfully
Number of chunks in vector database: 47


## Set up LLM and Retrieval

In [41]:
# Set up LLM and retrieval
local_model = "gemma2"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [42]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""Ты - AI-ассистент языковой модели. Твоя задача - сгенерировать 2
    разные версии заданного пользователем вопроса для поиска релевантных документов в
    векторной базе данных. Генерируя множественные перспективы на вопрос пользователя, твоя
    цель - помочь пользователю преодолеть некоторые ограничения основанного на расстоянии
    поиска по сходству. Предоставь эти альтернативные вопросы, разделенные новой строкой.
    Исходный вопрос: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [43]:
# RAG prompt template
template = """Отвечайте на вопрос ТОЛЬКО на основе следующего контекста:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [44]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [45]:
def chat_with_pdf(question):
    """
    Chat with the PDF using the RAG chain.
    """
    return display(Markdown(chain.invoke(question)))

In [46]:
# Example 1
chat_with_pdf("О чем этот документ?")

Документ содержит требования безопасности к разработке месторождений драгами и плавучими земснарядами.  

Также в документе упоминаются требования к электроаппаратуре, тормозным системам локомотивов и солекомбайнам с дизель-генераторной установкой. 


In [48]:
# Example 2 - mistral
chat_with_pdf("Требования безопасности к разработке месторождений")

Требования безопасности к разработке месторождений драгами и плавучими земснарядами. 

Рабочие места машиниста электросепаратора и оператора выпрямительных устройств должны быть оборудованы диэлектрическими изоляторами. 
Требования безопасности к переработке серных руд. 




In [23]:
# Example 3 llama3.2
chat_with_pdf("О чем этот документ?")

Этот document является Методическими рекомендациями по промышленной безопасности в шахтах и рудниках.

## Clean up (optional)

In [27]:
# Optional: Clean up when done 
vector_db.delete_collection()
print("Vector database deleted successfully")

Vector database deleted successfully
